<a href="https://colab.research.google.com/github/JungOhLee/LiTS_example/blob/main/UpsampleImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!pip install monai
!pip install nibabel
!pip install SimpleITK
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import SimpleITK as sitk
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

import nibabel
from monai.transforms import Resize

import time

In [11]:
from_dir = '/gdrive/MyDrive/LiTS_sample/preds_128/bonemeta_fn_0_3D_Unet_DropRes_lv5_All128_model_epoch100/numpy'
to_dir = '/gdrive/MyDrive/LiTS_sample/preds'
image_dir = '/gdrive/MyDrive/LiTS_sample/image'

os.makedirs(to_dir, exist_ok=True)

def get_from_path(file):
    return os.path.join(from_dir, file)

def get_image_path(file):
    file_path = file.split('.')[0] + '.nii'
    return os.path.join(image_dir, file_path)

def case_to_file(case):
    return case+'.npy'

def file_to_case(file_name):
    return file_name.split('.')[0]

In [12]:
import nibabel as nib

In [13]:
file_list = os.listdir(from_dir)
def get_orig_size(file_path):
  img_path = get_image_path(file_path)
  nifti_file = nib.load(img_path)
  img = nifti_file.get_fdata()
  return img.shape

for f in file_list:
  np_image = np.load(os.path.join(from_dir, f))
  target_shape = get_orig_size(f)
  resizer = Resize(target_shape)
  resized_image = resizer(np_image.transpose([2,1,0])[np.newaxis,:]) # with channel
  np.save(os.path.join(to_dir, f), resized_image[0]) # without channel